In [2]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option("display.max_columns", None)

data_modeling.h5 is the dataset merged and cleaned in the previous notebook.

In [3]:
store = pd.HDFStore('df_all_frac.h5')
data = store['df_all_frac']

In [3]:
data

,unique_id,loan_status,report_date,mob,bom,ppmt,ipmt,co_amt,prepay_amt,eom,prepay,default,loan_amount,monthly_payment,grade,income,inq_last_6m,num_open_accts,revol_bal,num_tot_accts,tot_credit_bal,accts_opn_last_6m,inq_last_12m,accts_opn_last_24m,employment_length,home_ownership,int_rate,earliest_cr_line,fico,util_rate,loan_over_income,term,dti,platform_id,Orig_year,Orig_month
0,A-953935-141664,current,SEP2018,23,5160.781280,346.245519,49.414481,0.0,0.0,4814.535760,0,0,12000.0,395.66,B,75000.0000,0.0,5.0,10620.0,17.0,10620.0,1.0,0.0,2.0,2,RENT,11.49,199.0,662.0,40.2,0.160000,36,0.0490,A,2016,10
1,A-598539-22454,current,AUG2018,17,4182.746349,190.750204,40.049796,0.0,0.0,3991.996146,0,0,7000.0,230.80,B,47500.0000,0.0,14.0,6875.0,16.0,29488.0,0.0,0.0,1.0,4,RENT,11.49,194.0,667.0,40.0,0.150000,36,0.2761,A,2017,3
2,A-55604-152970,current,FEB2017,1,16000.000000,333.220000,253.200000,0.0,0.0,15666.780000,0,0,16000.0,586.42,D,105000.0000,2.0,11.0,15114.0,13.0,181889.0,1.0,3.0,3.0,1,MORTGAGE/OWN,18.99,194.0,667.0,64.9,0.150000,36,0.1935,A,2017,1
3,A-535071-114597,current,FEB2019,2,11714.230000,288.230003,100.839997,0.0,0.0,11425.999997,0,0,12000.0,389.07,B,40000.0000,0.0,7.0,9202.0,12.0,33433.0,0.0,0.0,2.0,3,MORTGAGE/OWN,10.33,218.0,752.0,35.0,0.300000,36,0.2988,A,2018,12
4,A-585581-37352,current,JUN2018,10,17790.100138,259.657116,201.472884,0.0,0.0,17530.443023,0,0,20000.0,461.13,C,85000.0000,1.0,14.0,18910.0,18.0,554449.0,1.0,1.0,5.0,8,MORTGAGE/OWN,13.59,200.0,707.0,47.3,0.240000,60,0.3196,A,2017,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4344375,C-637041-9232,current,201707,11,17650.140000,255.340000,228.050000,0.0,0.0,17394.800000,0,0,20000.0,483.39,C,57600.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,NaN,15.72,NaN,710.0,NaN,0.347222,60,0.1500,C,2016,8
4344376,C-55844-4205,current,201603,1,35000.000000,494.330000,204.670000,0.0,0.0,34505.670000,0,0,35000.0,699.00,AA,76149.9996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,NaN,7.36,NaN,770.0,NaN,0.459619,60,0.3500,C,2016,2
4344377,C-208867-26393,current,201908,26,9666.290000,236.360000,84.560000,0.0,0.0,9429.930000,0,0,15000.0,320.92,B,68000.0004,2.0,9.0,6325.0,28.0,545865.0,1.0,2.0,2.0,6,MORTGAGE/OWN,10.30,33.0,750.0,31.0,0.220588,60,0.1800,C,2017,6
4344378,C-747566-31338,late_31_120_days,201910,26,1888.640000,0.000000,0.000000,0.0,0.0,1888.640000,0,0,5000.0,160.38,A,114999.9996,5.0,1.0,13333.0,12.0,38888.0,2.0,6.0,5.0,3,RENT,9.59,131.0,690.0,52.0,0.043478,36,0.1200,C,2017,8


Remove report_date

In [4]:
data.drop('report_date', inplace=True, axis=1)

Perform Treatment to the dti


In [5]:
data.loc[(data.platform_id == 'C'),'dti']=data['dti']-data['loan_over_income']
data['post_dti'] =  data['dti'] + data['loan_over_income']

In [6]:
X = data.drop(['default', 'prepay','unique_id', 'loan_status','bom','eom','ppmt','ipmt','prepay_amt','co_amt'], axis=1)
X_mob = X['mob']
X_year = X['Orig_year']
X_month = X['Orig_month']

X = X.drop('mob', axis=1)
X = X.drop('Orig_year', axis=1)
X = X.drop('Orig_month', axis=1)

y_default = data['default']
y_prepay = data['prepay']

In [7]:
encoder_1 = LabelEncoder()

# Fit the encoder on the data "platform_id"
encoder_1.fit(X["platform_id"])
X_platform_id_encoded = encoder_1.transform(X["platform_id"])
X["platform_id"] = X_platform_id_encoded

# Fit the encoder on the data "grade"
encoder_2 = LabelEncoder()
encoder_2.fit(X["grade"])
X_grade_encoded = encoder_2.transform(X["grade"])
X["grade"] = X_grade_encoded

In [8]:
# Fit the encoder on the data "home_ownership"
encoder_3 = LabelEncoder()
X_partial_home_ownership = X["home_ownership"].sample(frac=0.01)
encoder_3.fit(X_partial_home_ownership)
X_home_ownership_encoded = encoder_3.transform(X["home_ownership"])
X["home_ownership"] = X_home_ownership_encoded

In [9]:
# Do feature scaling
sc = StandardScaler()
X_scaled = sc.fit_transform(X.fillna(0))

# Apply PCA
pca = PCA(n_components= 11)
X_pca = pca.fit_transform(X_scaled)

# Get the transformed dataset
X_pca = pd.DataFrame(X_pca)
print(X_pca.head())
print("\nSize: ")
print(X_pca.shape)

         0         1         2         3         4         5         6   \
0 -0.962154  0.779693 -0.531650  1.163163 -0.682093 -0.045649 -0.699041   
1 -0.028577  0.512777 -3.078022 -0.486987  0.035361  0.674300  0.220458   
2  2.060993 -1.548725  0.307346 -1.974885  1.082134  1.242044 -1.556453   
3 -0.178743 -0.113873 -1.083110  1.937141  0.374389  0.737874 -0.047516   
4  2.233706  1.557031  3.643915  0.838310  0.445104 -0.501903  0.008767   

         7         8         9         10  
0  0.055255  0.162676 -0.467432  0.866656  
1 -0.045611  0.031261  0.353778  0.294199  
2 -0.163582  0.271928 -0.377169 -0.707195  
3 -0.125002  0.038776  0.684467 -1.059683  
4 -0.086495 -0.086662 -0.241188  0.137264  

Size: 
(10381413, 11)


In [10]:
store = pd.HDFStore('X_frac_pca.h5')
store['X_frac_pca'] = X_pca  # save it

store = pd.HDFStore('X_frac_mob.h5')
store['X_frac_mob'] = X_mob  # save it

store = pd.HDFStore('y_frac_default.h5')
store['y_frac_default'] = y_default  # save it

store = pd.HDFStore('y_frac_prepay.h5')
store['y_frac_prepay'] = y_prepay  # save it

store = pd.HDFStore('X_frac_year.h5')
store['X_frac_year'] = X_year  # save it

store = pd.HDFStore('X_frac_month.h5')
store['X_frac_month'] = X_month  # save it

